# Contents
1. Reproducibility
2. Loading Data
3. Validation Dataset
4. CNN
5. GPU Acceleration
6. Train
7. Save & Load
8. Transforming Image

# 1. Reproducibility
Please refer to the official document [REPRODUCIBILITY](https://pytorch.org/docs/stable/notes/randomness.html) for further information

Why can't I recreate the same results?

In [1]:
import os
import random
import numpy as np
import torch

In [2]:
SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Below could affect development or performance
torch.backends.cudnn.benckmark = False    # Disable cuDNN tuner to find the best algorithm for the hardware
torch.use_deterministic_algorithms(True)    # Force the use of deterministic algorithms, returning error for nondeterministic algorithms

# 2. Loading Data

In [3]:
import pandas as pd

In [4]:
def load_dataset(dir='./sample_data', train='mnist_train_small.csv', test='mnist_test.csv'):
    df_train = pd.read_csv(os.path.join(dir, train))
    df_test = pd.read_csv(os.path.join(dir, test))

    arr_train = df_train.to_numpy()
    arr_test = df_test.to_numpy()

    x_train = arr_train[:,1:]
    y_train = arr_train[:,:1]
    x_test = arr_test[:,1:]
    y_test = arr_test[:,:1]
    return x_train, x_test, y_train, y_test

In [5]:
x_train, x_test, y_train, y_test = load_dataset(dir='./sample_data', train='mnist_train_small.csv', test='mnist_test.csv')

xtr = torch.FloatTensor(x_train)
ytr = torch.LongTensor(y_train)
xte = torch.FloatTensor(x_test)
yte = torch.LongTensor(y_test)

ytr = ytr.squeeze()
yte = yte.squeeze()

In [6]:
print(xtr.shape)
print(ytr.shape)
print(xte.shape)
print(yte.shape)

torch.Size([19999, 784])
torch.Size([19999])
torch.Size([9999, 784])
torch.Size([9999])


## torch.utils.data.Dataset

In [7]:
from torch.utils.data import TensorDataset

In [8]:
train = TensorDataset(xtr, ytr)
test = TensorDataset(xte, yte)

In [9]:
sample_by_idx = train[777]
x, y = sample_by_idx

In [10]:
batch_by_idx = train[128:365]
x_batch, y_batch = batch_by_idx

### Custom Dataset

Define:

```python
__init__
__len__
__getitem__
```


In [11]:
from torch.utils.data import Dataset

In [12]:
class SmallMNIST(Dataset):
    def __init__(self, path):
        """
        To load and preprocess data
        """
        df = pd.read_csv(path)
        arr = df.to_numpy()

        xs = arr[:,1:]
        ys = arr[:,:1]

        self.x_data = torch.from_numpy(xs)
        self.y_data = torch.from_numpy(ys)

        self.y_data = self.y_data.type(torch.LongTensor)

    def __len__(self):
        """
        Return size
        """
        return len(self.x_data)

    def __getitem__(self, idx):
        """
        Indexing
        """
        return self.x_data[idx], self.y_data[idx]

In [13]:
train = SmallMNIST('./sample_data/mnist_train_small.csv')
test = SmallMNIST('./sample_data/mnist_test.csv')

In [14]:
epochs = 2
batch_size = 32

for epoch in range(epochs):
    for i in range(len(train) // batch_size + 1):
        xs, ys = train[i * batch_size:i * batch_size + batch_size]

## torch.utils.data.DataLoader

In [15]:
from torch.utils.data import DataLoader

In [16]:
train_loader = DataLoader(
    dataset=train,
    batch_size=4,
    shuffle=True,
    num_workers=0
)
test_loader = DataLoader(
    dataset=test,
    batch_size=4,
    shuffle=False,
    num_workers=0
)

In [17]:
dataiter = iter(train_loader)
sample = dataiter.next()
features, labels = sample
print(features.shape, labels.shape)

torch.Size([4, 784]) torch.Size([4, 1])


In [18]:
epochs = 2

for epoch in range(epochs):
    for xs, ys in train:
        pass

# 3. Validation Dataset

## Index slicing

In [19]:
x_train, y_train = xtr[:15000], ytr[:15000]
x_valid, y_valid = xtr[15000:], ytr[15000:]

In [20]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=8)

## torch.utils.data.random_split

In [21]:
tr, val = torch.utils.data.random_split(train, [int(len(train) * 0.8) + 1, int(len(train) * 0.2)])

print(tr)
print(len(tr))
print(val)
print(len(val))

16000
3999


In [22]:
train_loader = DataLoader(
    dataset=tr,
    batch_size=4,
    shuffle=True,
    num_workers=0
)
val_loader = DataLoader(
    dataset=val,
    batch_size=4,
    shuffle=False,
    num_workers=0,
)

## torch.utils.data.SubsetRandomSampler

In [23]:
from torch.utils.data.sampler import SubsetRandomSampler

In [24]:
indices = list(range(len(train)))
np.random.shuffle(indices)
print(indices)

[10650, 2041, 8668, 1114, 13902, 11963, 11072, 3002, 19770, 8115, 3525, 7879, 16441, 16296, 12566, 5252, 19392, 16796, 8094, 322, 9923, 7459, 19023, 17552, 9119, 16421, 16060, 10120, 19054, 3649, 3941, 19846, 16675, 12630, 11443, 12157, 13835, 3434, 18738, 17406, 15440, 10406, 6517, 971, 16487, 19729, 13137, 3669, 15983, 4058, 14976, 15698, 13829, 13419, 960, 17743, 9864, 11263, 15782, 17196, 7554, 10498, 17059, 17927, 7085, 14753, 4519, 3024, 12121, 18493, 5483, 1419, 6209, 10742, 16143, 17616, 14568, 980, 2783, 7713, 11518, 8600, 11640, 18829, 12562, 12751, 6384, 4807, 6195, 10648, 7567, 16397, 17523, 1346, 13124, 12673, 15455, 14001, 18620, 2119, 15304, 4309, 4484, 11013, 4770, 2712, 12186, 16615, 10947, 2545, 12619, 9851, 4410, 19646, 12344, 12407, 9610, 10718, 7571, 3570, 8602, 3559, 4991, 7439, 9325, 2300, 7666, 5804, 12022, 15537, 9757, 12014, 5211, 11548, 12366, 9270, 3975, 16090, 17232, 17442, 13656, 15645, 2730, 3298, 8956, 17409, 15155, 17276, 6703, 1968, 4936, 2112, 878, 19

In [25]:
split = int(np.floor(0.8 * len(train)))
tr_idx, val_idx = indices[:split], indices[split:]

In [26]:
tr_sampler = SubsetRandomSampler(tr_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [27]:
train_loader = DataLoader(
    dataset=train,
    batch_size=4,
    num_workers=0,
    sampler=tr_sampler
)
val_loader = DataLoader(
    dataset=train,
    batch_size=4,
    num_workers=0,
    sampler=val_sampler
)

## sklearn.model_selection.train_test_split

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
tr_idx, val_idx = train_test_split(list(range(len(train))), test_size=0.2, random_state=SEED, stratify=ytr.numpy())

In [30]:
tr_sampler = SubsetRandomSampler(tr_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [31]:
train_loader = DataLoader(
    dataset=train,
    batch_size=4,
    num_workers=0,
    sampler=tr_sampler
)
val_loader = DataLoader(
    dataset=train,
    batch_size=4,
    num_workers=0,
    sampler=val_sampler
)

# 4. Convoluational Neural Networks

In [32]:
import torch.nn as nn
import torch.nn.functional as F

In [33]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=1)
        self.fc1 = nn.Linear(1152, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = self.conv1(x)
        x = F.relu(x)
        x = F.avg_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = torch.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        logits = self.fc2(x)
        return logits

Remember:
```
(N - F + 2P) / S + 1
```

# 5. GPU Acceleration

In [34]:
torch.cuda.is_available()

True

In [35]:
torch.cuda.device_count()

1

In [36]:
torch.cuda.current_device()

0

In [37]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [38]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [39]:
a = torch.tensor([1, 2], device=device)
b = torch.tensor([3, 4])
bcuda = b.cuda()
c = torch.tensor([5, 6]).to(device)

In [40]:
print(a + bcuda)

tensor([4, 6], device='cuda:0')


In [41]:
print(a + b)

RuntimeError: ignored

In [42]:
torch.cuda.empty_cache()

# 6. Train

## Set Device

In [43]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Prepare Data

In [44]:
class SmallMNIST(Dataset):
    def __init__(self, path):
        df = pd.read_csv(path)
        arr = df.to_numpy()

        xs = arr[:,1:]
        ys = arr[:,:1]

        self.x_data = torch.from_numpy(xs)
        self.y_data = torch.from_numpy(ys)
        self.y_data.squeeze_()

        self.x_data = self.x_data.type(torch.FloatTensor)
        self.y_data = self.y_data.type(torch.LongTensor)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

In [45]:
train = SmallMNIST('./sample_data/mnist_train_small.csv')
test = SmallMNIST('./sample_data/mnist_test.csv')

In [46]:
tr_idx, val_idx = train_test_split(list(range(len(train))), test_size=0.2, random_state=SEED, stratify=train.y_data.numpy())

In [47]:
tr_sampler = SubsetRandomSampler(tr_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [48]:
train_loader = DataLoader(
    dataset=train,
    batch_size=4,
    num_workers=0,
    sampler=tr_sampler
)
val_loader = DataLoader(
    dataset=train,
    batch_size=4,
    num_workers=0,
    sampler=val_sampler
)
test_loader = DataLoader(
    dataset=test,
    batch_size=4,
    num_workers=0,
)

## Define Model, Loss, Optimizer

In [49]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(288, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 288)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [50]:
model = Net()
model = model.to(device)

In [51]:
import torch.optim as optim

In [52]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Training

In [ ]:
epochs = 2

for epoch in range(epochs):

    train_loss = 0
    train_total = 0
    train_correct = 0
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
    train_acc = train_correct / train_total

    valid_loss = 0
    valid_total = 0
    valid_correct = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, labels)
            valid_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            valid_total += labels.size(0)
            valid_correct += (predicted == labels).sum().item()
    valid_acc = valid_correct / valid_total

    print('Epoch: ', epoch + 1)
    print('Loss: ', 'Train %.4f / Valid %.4f' % (train_loss, valid_loss))
    print('Accuracy: ', 'Train %.4f / Valid %.4f' % (train_acc * 100, valid_acc * 100))

Epoch:  1
Loss:  Train 1789.3115 / Valid 413.7367
Accuracy:  Train 88.9681 / Valid 90.1250
Epoch:  2
Loss:  Train 2027.9637 / Valid 512.7833
Accuracy:  Train 88.1993 / Valid 88.4250
Epoch:  3
Loss:  Train 2230.4193 / Valid 520.3309
Accuracy:  Train 87.3867 / Valid 86.6500
Epoch:  4
Loss:  Train 2606.0506 / Valid 652.8495
Accuracy:  Train 85.5866 / Valid 87.3750
Epoch:  5
Loss:  Train 3499.0681 / Valid 765.6300
Accuracy:  Train 80.7175 / Valid 78.2000
Epoch:  6
Loss:  Train 3453.4325 / Valid 707.6563
Accuracy:  Train 80.0925 / Valid 82.3250
Epoch:  7
Loss:  Train 3895.7903 / Valid 1250.9303
Accuracy:  Train 77.4798 / Valid 79.4750
Epoch:  8
Loss:  Train 3889.7453 / Valid 929.2280
Accuracy:  Train 76.9673 / Valid 72.2250
Epoch:  9
Loss:  Train 5042.6691 / Valid 1386.3053
Accuracy:  Train 65.9104 / Valid 61.0000
Epoch:  10
Loss:  Train 8193.8253 / Valid 2306.6379
Accuracy:  Train 30.6582 / Valid 10.1000
Epoch:  11
Loss:  Train 9214.1385 / Valid 2300.8462
Accuracy:  Train 10.7194 / Valid 1

# 7. Save & Load

In [54]:
torch.save(model.state_dict(), './model.pt')

In [55]:
print('state_dict format of the model: {}'.format(model.state_dict()))

state_dict format of the model: OrderedDict([('conv1.weight', tensor([[[[-1.2712e-01,  2.0949e-02, -3.2422e-01, -2.3098e-01, -5.2735e-02],
          [ 7.0722e-02, -3.0851e-03,  1.7839e-02, -2.4623e-01,  9.3926e-02],
          [-2.2392e-01, -1.3677e-01, -2.9930e-01, -2.0662e-01,  1.4319e-01],
          [-1.0287e-01, -9.8916e-02, -2.1496e-02,  8.7212e-02, -1.5972e-01],
          [-2.7021e-01, -1.9763e-01, -1.5417e-01, -1.0464e-01, -7.7799e-02]]],


        [[[ 8.3111e-02, -5.2242e-01, -1.7344e-01, -1.4080e-01, -1.6534e-01],
          [ 3.2037e-01, -2.9874e-01, -8.9098e-02, -2.1436e-01,  3.1502e-03],
          [ 2.0998e-01, -4.3892e-01, -8.1599e-02, -1.0169e-01, -1.1662e-01],
          [ 1.8925e-01, -3.4853e-01,  1.4661e-03, -4.3726e-02, -9.3879e-02],
          [-2.6186e-01, -2.1346e-01, -2.8011e-01, -1.3517e-01, -2.3463e-02]]],


        [[[ 1.4936e-02,  5.1427e-02,  2.7186e-01,  3.5006e-01, -1.3086e-01],
          [-3.5961e-01,  3.3522e-02,  3.5818e-02,  5.4177e-02, -5.8993e-01],
      

In [56]:
new_model = Net()
new_model.load_state_dict(torch.load('./model.pt'))

<All keys matched successfully>

# 8. Transforming Image

## torchvision.transforms

In [ ]:
!wget https://i.pinimg.com/originals/21/ae/ae/21aeae8d2cae56289bd55665fb5d9b8f.jpg
!mv 21aeae8d2cae56289bd55665fb5d9b8f.jpg example.jpg
!rm 21aeae8d2cae56289bd55665fb5d9b8f.jpg

In [ ]:
from IPython.display import Image, display

display(Image(filename='example.jpg'))

In [ ]:
from PIL import Image as Img

im = Img.open('example.jpg')

print(im.size)
type(im)

In [ ]:
from torchvision import transforms

In [ ]:
trans = transforms.ToTensor()

x = trans(im)

In [ ]:
print(x.size())

In [ ]:
trans = transforms.Resize((256, 384))

resized = trans(im)

display(resized)
print(resized.size)

In [ ]:
trans = transforms.CenterCrop((100, 150))

newim = trans(resized)

display(newim)

In [ ]:
trans = transforms.RandomCrop((100, 100), padding=30, padding_mode='constant')

newim = trans(resized)

display(newim)

In [ ]:
trans = transforms.Grayscale()

newim = trans(resized)

display(newim)

In [ ]:
trans = transforms.ToTensor()

print(trans(newim).size())

In [ ]:
trans = transforms.ColorJitter(brightness=0, contrast=3, saturation=6, hue=0.3)

newim = trans(resized)

display(newim)

In [ ]:
trans = transforms.RandomVerticalFlip(p=1)

newim = trans(resized)

display(newim)

In [ ]:
trans = transforms.RandomRotation(degrees=40, center=(40, 70))

newim = trans(resized)

display(newim)

In [ ]:
trans = transforms.RandomAffine(degrees=0, translate=(0.2, 0.8))

newim = trans(resized)

display(newim)

In [ ]:
trans = transforms.Resize((4, 6))

resized = trans(im)

trans = transforms.ToTensor()

x = trans(resized)

print(x[:2, :2])

In [ ]:
trans = transforms.Normalize(mean=(0.5, 0.3, 0.2), std=(0.5, 0.4, 0.5))

newx = trans(x)

print(newx[:2, :2])

In [ ]:
trans = transforms.RandomErasing(p=1, scale=(0.1, 0.3))

newx = trans(x)

print(newx[:2, :2])

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(),
    transforms.RandomAffine(degrees=20, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5)),
])

## torchvision.datasets

In [ ]:
import torchvision.datasets as datasets

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

train_dataset = torchvision.datasets.MNIST(
    root='./',
    download=True,
    train=True,
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='./',
    download=False,
    train=False,
    transform= transform
)